In [2]:
import pandas as pd

# Import MTeams.csv
teams_df = pd.read_csv("MTeams.csv")

# Import MSeasons.csv
seasons_df = pd.read_csv("MSeasons.csv")

# Import MNCAATourneySeeds.csv
seeds_df = pd.read_csv("MNCAATourneySeeds.csv")

# Import MRegularSeasonCompactResults.csv
compact_results_df = pd.read_csv("MRegularSeasonCompactResults.csv")

# Import MRegularSeasonDetailedResults.csv
detailed_results_regular_df = pd.read_csv("MRegularSeasonDetailedResults.csv")

detailed_results_tourney_df = pd.read_csv("MNCAATourneyDetailedResults.csv")
# Import MMasseyOrdinals.csv
massey_ordinals_df = pd.read_csv("MMasseyOrdinals.csv")

# Import MTeamCoaches.csv
team_coaches_df = pd.read_csv("MTeamCoaches.csv")

In [3]:
unc_team_id = teams_df.loc[
    teams_df["TeamName"].str.contains("UNC Asheville"), "TeamID"
].values[0]

print(unc_team_id)

1421


In [4]:
teams_df[teams_df["TeamID"] == 1421]

,TeamID,TeamName,FirstD1Season,LastD1Season
320,1421,UNC Asheville,1987,2024


In [5]:
unc_coaches_df = team_coaches_df[team_coaches_df["TeamID"] == unc_team_id]

In [6]:
unc_coaches_df["CoachName"].value_counts()

CoachName
eddie_biedenbach     17
mike_morell           6
don_doucette          5
nicholas_mcdevitt     5
randy_wiel            3
jerry_green           2
Name: count, dtype: int64

In [7]:
unc_results_df = detailed_results_regular_df[
    (detailed_results_regular_df["WTeamID"] == 1421)
    | (detailed_results_regular_df["LTeamID"] == 1421)
]

In [8]:
combined_df = unc_results_df.merge(
    unc_coaches_df[["Season", "CoachName"]], on="Season", how="left"
)


combined_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,CoachName
0,2003,18,1277,66,1421,52,H,0,25,51,...,6,6,7,17,13,16,11,1,21,eddie_biedenbach
1,2003,20,1278,87,1421,81,H,0,27,57,...,17,24,18,24,12,15,4,4,27,eddie_biedenbach
2,2003,28,1421,77,1441,72,H,0,26,59,...,17,23,12,26,10,12,5,3,22,eddie_biedenbach
3,2003,33,1190,87,1421,69,H,0,33,70,...,14,17,11,22,12,20,6,5,17,eddie_biedenbach
4,2003,40,1421,99,1144,89,H,1,33,68,...,13,22,11,23,21,17,7,4,21,eddie_biedenbach


In [9]:
combined_df["CoachName"].value_counts()

CoachName
eddie_biedenbach     307
mike_morell          167
nicholas_mcdevitt    149
Name: count, dtype: int64

In [16]:
import pandas as pd

# Assuming combined_df is your existing DataFrame
# Create an empty list to store dictionaries
data = []

# Filter rows where WTeamID or LTeamID is 1421
team_1231_df = combined_df[
    (combined_df["WTeamID"] == 1421) | (combined_df["LTeamID"] == 1421)
]

# Iterate over each row
for index, row in team_1231_df.iterrows():
    if row["WTeamID"] == 1421:
        # Store values of columns starting with 'W' and rename them
        values = {
            col[1:]: row[col] for col in team_1231_df.columns if col.startswith("W")
        }
        # Add coach name column
        values["CoachName"] = row["CoachName"]
    else:
        # Store values of columns starting with 'L' and rename them
        values = {
            col[1:]: row[col] for col in team_1231_df.columns if col.startswith("L")
        }
        # Add coach name column
        values["CoachName"] = row["CoachName"]

    # Append the values to the list
    data.append(values)

# Create a new DataFrame from the list of dictionaries
new_df = pd.DataFrame(data)

In [17]:
new_df.head()

,TeamID,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,CoachName,Loc
0,1421,52,20,49,6,23,6,6,7,17,13,16,11,1,21,eddie_biedenbach,NaN
1,1421,81,28,63,8,16,17,24,18,24,12,15,4,4,27,eddie_biedenbach,NaN
2,1421,77,26,59,5,17,20,29,12,32,16,12,8,4,21,eddie_biedenbach,H
3,1421,69,24,57,7,15,14,17,11,22,12,20,6,5,17,eddie_biedenbach,NaN
4,1421,99,33,68,11,19,22,27,15,28,21,18,11,4,20,eddie_biedenbach,H


In [9]:
# Import the necessary libraries
from sklearn.preprocessing import OneHotEncoder

# Create an instance of the OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Reshape the CoachName column to a 2D array
coach_names = new_df["CoachName"].values.reshape(-1, 1)

# Encode the CoachName column using one-hot encoding
coach_names_encoded = encoder.fit_transform(coach_names)

# Create a DataFrame from the encoded CoachName column
coach_names_encoded_df = pd.DataFrame(
    coach_names_encoded, columns=encoder.categories_[0]
)

# Concatenate the encoded CoachName DataFrame with the original combined_df
combined_df_encoded = pd.concat([new_df, coach_names_encoded_df], axis=1)

# Remove the original CoachName column from the combined_df_encoded
combined_df_encoded.drop("CoachName", axis=1, inplace=True)

c:\Users\ayush\miniconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import time
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = new_df.select_dtypes(include="number")

# Assign the 'CoachName' column as the target variable
y = new_df["CoachName"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)

# Create a decision tree classifier
dt_model = DecisionTreeClassifier(random_state=42)

param_dist = {
    "max_depth": [3, 5, 7, 9],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "criterion": ["gini", "entropy"],
}


# Create a randomized search object
random_search_dt = RandomizedSearchCV(
    estimator=dt_model,
    param_distributions=param_dist,
    n_iter=100,
    cv=5,
    n_jobs=-1,
    random_state=42,
)

# Start timing
start_time = time.time()

# Perform randomized search to find the best parameters
with tqdm(total=random_search_dt.n_iter, desc="Parameter optimization") as pbar:
    random_search_dt.fit(X_train, y_encoded)
    pbar.update(1)

# End timing
end_time = time.time()
total_time = end_time - start_time

# Print the best parameters found
print("Best parameters:", random_search_dt.best_params_)

# Get the best model
best_model_dt = random_search_dt.best_estimator_

# Make predictions on the testing data using the best model
y_pred_dt = best_model_dt.predict(X_test)

# Decode the predicted labels back to their original form
y_pred_decoded_dt = label_encoder.inverse_transform(y_pred_dt)

# Calculate the accuracy of the best model
accuracy_dt = accuracy_score(y_test, y_pred_decoded_dt)
print("Accuracy:", accuracy_dt)
print("Total time for tuning and modeling:", total_time, "seconds")

importance_scores = best_model_dt.feature_importances_

Parameter optimization:   0%|          | 0/100 [00:00<?, ?it/s]c:\Users\ayush\miniconda3\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 72 is smaller than n_iter=100. Running 72 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
Parameter optimization:   1%|          | 1/100 [00:05<09:45,  5.91s/it]

Best parameters: {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 3, 'criterion': 'gini'}
Accuracy: 0.568
Total time for tuning and modeling: 5.918956518173218 seconds


In [20]:
from sklearn.tree import DecisionTreeClassifier

importance_scores
# Create a DecisionTreeClassifier object
clf = DecisionTreeClassifier()

# Fit the model on your data
clf.fit(X, y)

# Get the feature importances
importances = clf.feature_importances_

# Create a dictionary of feature names and their importances
feature_importances = dict(zip(X.columns, importance_scores))

# Sort the features by importance in descending order
sorted_features = sorted(feature_importances.items(), key=lambda x: x[1], reverse=True)

# Print the sorted features
for feature, importance in sorted_features:
    print(f"{feature}: {importance}")

OR: 0.4355721670710489
FGA3: 0.17613633547514979
Ast: 0.14369185837027748
PF: 0.11253461708543075
Score: 0.08033691818975232
FTA: 0.05172810380834078
TeamID: 0.0
FGM: 0.0
FGA: 0.0
FGM3: 0.0
FTM: 0.0
DR: 0.0
TO: 0.0
Stl: 0.0
Blk: 0.0
